# Datasets

> Download and store datasets

In [ ]:
#| default_exp datasets

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import os
from pathlib import Path

from pooch import create as pooch_create, retrieve as pooch_retrieve, Decompress, Unzip, Untar
from sklearn.model_selection import train_test_split
import quilt3
import pandas as pd
import numpy as np
from PIL import Image
import tifffile as tiff
from tqdm import tqdm

import medmnist


### MedMNIST Datasets

In [ ]:
#| export
def download_medmnist(dataset: str, # The name of the MedMNIST dataset (e.g., 'pathmnist', 'bloodmnist', etc.).
                      output_dir: str = '.', # The path to the directory where the datasets will be saved.
                      download_only: bool = False, # If True, only download the dataset into the output directory without processing.
                      save_images: bool = True, # If True, save the images into the output directory as .png (2D datasets) or multipage .tiff (3D datasets) files.
                      ):
    """
    Downloads the specified MedMNIST dataset and saves the training, validation, and test datasets 
    into the specified output directory. Images are saved as .png for 2D data and multi-page .tiff for 3D data,
    organized into folders named after their labels.

    Returns: None, saves images in the specified output directory if save_images is True.
    """

    # Check if the dataset is available in the MedMNIST information dictionary
    if dataset not in medmnist.INFO:
        raise ValueError(f"The dataset '{dataset}' is not available. Please select from the available datasets.")

    # Retrieve dataset information
    info = medmnist.INFO[dataset]

    # Get the appropriate dataset class from MedMNIST
    dataset_class = getattr(medmnist, info['python_class'])
    
    # Check if the dataset has already been downloaded and processed
    dataset_path = os.path.join(output_dir, dataset)
    if os.path.exists(dataset_path) and len(os.listdir(dataset_path)) > 0:
        print(f"Dataset '{dataset}' is already downloaded and available in '{dataset_path}'.")
        if download_only:
            return info
        elif save_images:
            print("Skipping download and image saving, as data already exists.")
            return None

    # Create the output directory if it doesn't exist
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)

    # Download the datasets
    train_dataset = dataset_class(split='train', download=True, root=dataset_path)
    val_dataset = dataset_class(split='val', download=True, root=dataset_path)
    test_dataset = dataset_class(split='test', download=True, root=dataset_path)

    # Save the images into directories by their label
    def save_images(dataset, split):
        """Helper function to save images and labels into directories."""
        split_dir = os.path.join(dataset_path, split)
        if not os.path.exists(split_dir):
            os.makedirs(split_dir)

        for i in tqdm(range(len(dataset))):
            img, label = dataset[i]
            label_dir = os.path.join(split_dir, str(label).replace("[", "").replace("]", ""))  # Remove parentheses
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)

            # Save 2D images as .png
            if info['n_channels'] == 1:  
                if img.ndim == 3:   # Check if it's 2D (single-channel)
                    img_path = os.path.join(label_dir, f'{split}_{i}.png')
                    img = Image.fromarray(img.squeeze(), mode='L')  # 'L' mode for grayscale
                    img.save(img_path)
                elif img.ndim == 4:
                    img_path = os.path.join(label_dir, f'{split}_{i}.tiff')
                    tiff.imwrite(img_path, img)
            elif info['n_channels'] == 3:  # Check if it's RGB
                img_path = os.path.join(label_dir, f'{split}_{i}.png')
                img.save(img_path)
            # Save 3D images as multi-page .tiff
            else:
                img_path = os.path.join(label_dir, f'{split}_{i}.tiff')
                tiff.imwrite(img_path, img)

    # Save training, validation, and test data if save_images is True
    if save_images:
        print(f"Saving training images to {dataset_path}...")
        save_images(train_dataset, 'train')

        print(f"Saving validation images to {dataset_path}...")
        save_images(val_dataset, 'val')

        print(f"Saving test images to {dataset_path}...")
        save_images(test_dataset, 'test')
        
        # Clean up: remove .npz files if present
        for file in os.listdir(dataset_path):
            if file.endswith('.npz'):
                os.remove(os.path.join(dataset_path, file))
                print(f"Removed {file}")

    # If download_only is True, skip returning the dataset objects and just download the files
    if download_only:
        print(f"Datasets downloaded to {dataset_path}")
        print(f"Dataset info for '{dataset}': {info}")
        return info

    # Return the datasets if download_only is False and save_images is False
    return train_dataset, val_dataset, test_dataset if not save_images else None


In [ ]:
#| export
def medmnist2df(train_dataset,     # MedMNIST training dataset with images and labels
                val_dataset=None,  # (Optional) MedMNIST validation dataset with images and labels
                test_dataset=None, # (Optional) MedMNIST test dataset with images and labels
                mode='RGB'         # Mode for PIL Image conversion, e.g., 'RGB', 'L'
               ) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame): # (df_train, df_val, df_test): DataFrames with columns 'image' and 'label'
    """
    Convert MedMNIST datasets to DataFrames, with images as PIL Image objects and labels as DataFrame columns.
    
    Missing datasets (if None) are represented by None in the return tuple.
    """
    
    # Helper function to convert dataset to DataFrame
    def dataset_to_df(dataset, mode):
        images, labels = dataset.imgs, dataset.labels
        return pd.DataFrame({
            'image': [Image.fromarray(img, mode) for img in images], 
            'label': labels.squeeze()
        })
    
    # Convert each dataset to a DataFrame if it exists
    df_train = dataset_to_df(train_dataset, mode) if train_dataset is not None else None
    df_val = dataset_to_df(val_dataset, mode) if val_dataset is not None else None
    df_test = dataset_to_df(test_dataset, mode) if test_dataset is not None else None
    
    return df_train, df_val, df_test


### Download data via Pooch

In [ ]:
#| export
def download_file(url, # The URL of the file to be downloaded
                  output_dir="data", # The directory where the downloaded file will be saved
                  extract=True, # If True, decompresses the file if it's in a compressed format
                  hash=None, # Optional: You can add a checksum for integrity verification
                  extract_dir=None, # Directory to extract the files to
                  ):
    """
    Download and optionally decompress a single file using Pooch.

    """
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Set processor to Decompress if extract=True
    processor = None
    if extract:
        if url.endswith('.zip'):
            processor = Unzip(extract_dir=extract_dir)
        elif url.endswith(('.tar', '.tar.gz', '.tar.bz2', '.tar.xz')):
            processor = Untar(extract_dir=extract_dir)
        elif url.endswith(('.gz', '.bz2', '.xz')):
            processor = Decompress()

    # Download the file, decompressing if extract=True and file is compressed
    downloaded_file = pooch_retrieve(
        url=url,
        path=output_dir,
        processor=processor,
        known_hash=hash,  # Optional: You can add a checksum for integrity verification
    )
    
    print("The file has been downloaded and saved to:", os.path.abspath(output_dir))
    # Get the name of the extraction subdirectory if extract = True
    if extract:
        output_dir_len = len(Path(os.path.abspath(output_dir)).parts)
        file0 = os.path.abspath(downloaded_file[0])
        extract_dir_abs = os.path.join(*Path(file0).parts[0:output_dir_len+1])
        print("Extracted files have been saved to:", extract_dir_abs)
        return extract_dir_abs

In [ ]:
#| export
def download_dataset(base_url, # The base URL from which the files will be downloaded.
                     expected_checksums, #  A dictionary mapping file names to their expected checksums.
                     file_names, # A dictionary mapping task identifiers to file names.
                     output_dir, # The directory where the downloaded files will be saved.
                     processor=None, # A function to process the downloaded data.
                     ):
    """
    Download a dataset using Pooch and save it to the specified output directory.

    """
    # Create a Pooch object with the base URL, output directory, and expected checksums
    pooch_instance = pooch_create(
        path=output_dir,
        base_url=base_url,
        registry=expected_checksums,
    )
    
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Download each file if it is not already present in the output directory
    for _, file_name in file_names.items():
        pooch_instance.fetch(file_name, progressbar=True, processor=processor)
    
    print("The dataset has been successfully downloaded and saved to:", output_dir)


In [ ]:
#| hide

# # Specify the directory where you want to save the downloaded files
output_directory = "./_test_folder"

# Define the base URL for the MSD dataset
base_url = 'https://s3.ap-northeast-1.wasabisys.com/gigadb-datasets/live/pub/10.5524/100001_101000/100888/'

# Define the expected checksums for the files in the dataset
expected_checksums = {
'Samples/actin-20x-noise1-highsnr-sample.png': 'md5:7995383f95473a4e74a3b49ed2d6a846'
}

# Define the names of the files to be downloaded
file_names = {
'1': 'Samples/actin-20x-noise1-highsnr-sample.png'
}

# Download the dataset
download_dataset(base_url, expected_checksums, file_names, output_directory)










100%|█████████████████████████████████████| 2.38M/2.38M [00:00<00:00, 1.75GB/s]

The dataset has been successfully downloaded and saved to: ./_test_folder


In [ ]:
#| export
def download_dataset_from_csv(csv_file, # Path to the CSV file containing file names and checksums.
                              base_url, # The base URL from which the files will be downloaded.
                              output_dir, # The directory where the downloaded files will be saved.
                              processor=None, # A function to process the downloaded data.
                              rows=None, # Specific row indices to download. If None, download all rows.
                              prepend_mdf5=True, # If True, prepend 'md5:' to the checksums.
                              ):
    """
    Download a dataset using Pooch and save it to the specified output directory, reading file names and checksums from a CSV file.

    """
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # If specific rows are provided, filter the DataFrame
    if rows is not None:
        df = df.iloc[rows]
        
    # Create a dictionary for expected checksums
    if prepend_mdf5:
        expected_checksums = pd.Series(df['MD5 Checksum'].apply(lambda x: f"md5:{x}").values, index=df['Filename']).to_dict()
    else:
        expected_checksums = pd.Series(df['MD5 Checksum'].values, index=df['Filename']).to_dict()

    # Create a Pooch object with the base URL, output directory, and expected checksums
    pooch_instance = pooch_create(
        path=output_dir,
        base_url=base_url,
        registry=expected_checksums,
    )
    
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Download each file if it is not already present in the output directory
    for file_name in df['Filename']:
        pooch_instance.fetch(file_name, progressbar=True, processor=processor)
    
    print("The dataset has been successfully downloaded and saved to:", output_dir)



In [ ]:
# Specify the directory where you want to save the downloaded files
output_directory = "./_test_folder"
# Define the base URL for the MSD dataset
base_url = 'https://s3.ap-northeast-1.wasabisys.com/gigadb-datasets/live/pub/10.5524/100001_101000/100888/'

download_dataset_from_csv('./data_examples/FMD_dataset_info.csv', base_url, output_directory, rows=[6])


The dataset has been successfully downloaded and saved to: ./_test_folder


### Download data via Quilt/T4

Allen Institute Cell Science (AICS)

In [ ]:
#| export

def aics_pipeline(n_images_to_download=40, # Number of images to download
                  image_save_dir=None, # Directory to save the images
                  col="SourceReadPath", # Column name for image paths in the data manifest
                  ): 
    # Set default save directory if not provided
    if image_save_dir is None:
        image_save_dir = os.getcwd()

    # Ensure the save directory exists; create it if not
    os.makedirs(image_save_dir, exist_ok=True)

    # Load the package
    package = quilt3.Package.browse("aics/pipeline_integrated_cell", registry="s3://allencell")
    
    # Load metadata
    data_manifest = package["metadata.csv"]()

    # Keep only unique FOVs
    unique_fov_indices = np.unique(data_manifest['FOVId'], return_index=True)[1]
    data_manifest = data_manifest.iloc[unique_fov_indices]

    # Select first n_images_to_download
    data_manifest = data_manifest.iloc[:n_images_to_download]

    # Get source and target paths
    image_source_paths = data_manifest[col]
    image_target_paths = [os.path.join(image_save_dir, os.path.basename(image_source_path)) 
                          for image_source_path in image_source_paths]

    # Download images
    downloaded_image_paths = []
    for image_source_path, image_target_path in zip(image_source_paths, image_target_paths):
        if os.path.exists(image_target_path):
            downloaded_image_paths.append(image_target_path)
            continue  # Skip if already downloaded
        
        try:
            package[image_source_path].fetch(image_target_path)
            downloaded_image_paths.append(image_target_path)
        except (OSError, FileNotFoundError) as e:
            print(f"Failed to fetch {image_source_path}: {e}")
                
    return downloaded_image_paths, data_manifest


In [ ]:
image_target_paths, data_manifest = aics_pipeline(1, "../_data/aics")


Loading manifest: 100%|██████████| 77165/77165 [00:01<00:00, 45.2k/s]


In [ ]:
print(image_target_paths)
data_manifest.to_csv('../_data/aics/aics_dataset.csv')

['../_data/aics/9e5d8f2e_3500001004_100X_20170623_5-Scene-1-P24-E06.czi_nucWholeIndexImageScale.tiff', '../_data/aics/77a69ff1_3500001004_100X_20170623_5-Scene-3-P26-F05.czi_nucWholeIndexImageScale.tiff']


In [ ]:
image_target_paths, data_manifest = aics_pipeline(1, "../_data/aics", col="NucleusSegmentationReadPath")


Loading manifest: 100%|██████████| 77165/77165 [00:01<00:00, 46.5k/s]
100%|██████████| 491k/491k [00:02<00:00, 171kB/s] 


### Dataset Manifest

Utilities to make a list of all of the files of the train and test dataset in csv form. 

In [ ]:
#| export
def manifest2csv(signal,                # List of paths to signal images
                 target,                # List of paths to target images
                 paths=None,            # List of paths to images
                 train_fraction=0.8,    # Fraction of data to use for training
                 data_save_path='./',   # Path to save the CSV files
                 train='train.csv',     # Name of the training CSV file
                 test='test.csv',       # Name of the test CSV file
                 identifier=None,       # Identifier to add to the paths
                 ):
    
    if paths is None:
        df = pd.DataFrame(columns=["path_signal", "path_target"])
        df["path_signal"] = signal
        df["path_target"] = target 
        length_dataset = len(signal)
    elif identifier is None:
        df = pd.DataFrame(columns=["path_tiff", "channel_signal", "channel_target"])
        df["path_tiff"] = paths
        df["channel_signal"] = signal
        df["channel_target"] = target 
        length_dataset = len(paths)
    else:
        df = pd.DataFrame(columns=["signal (path+identifier)", "target (path+identifier)"])
        df["signal (path+identifier)"] = paths
        df["signal (path+identifier)"] = df["signal (path+identifier)"] + identifier + [*signal.astype(str)]
        df["target (path+identifier)"] = paths
        df["target (path+identifier)"] = df["target (path+identifier)"] + identifier + [*target.astype(str)]       
        length_dataset = len(paths)
        

    n_train_images = int(length_dataset * train_fraction)
    df_train = df[:n_train_images]
    df_test = df[n_train_images:]

    df_test.to_csv(data_save_path+test, index=False)
    df_train.to_csv(data_save_path+train, index=False)

In [ ]:

manifest2csv(data_manifest["ChannelNumberBrightfield"],data_manifest["ChannelNumber405"], image_target_paths, data_save_path='./data_examples/')


In [ ]:
#| export
def split_dataframe(input_data, # Path to CSV file or DataFrame
                    train_fraction=0.7, # Proportion of data to use for the training set
                    valid_fraction=0.1, # Proportion of data to use for the validation set
                    split_column=None, # Column name that indicates pre-defined split
                    stratify=False, # If True, stratify by split_column during random split
                    add_is_valid=False, # If True, adds 'is_valid' column in the train set to mark validation samples
                    train_path="train.csv", # Path to save the training CSV file
                    test_path="test.csv", # Path to save the test CSV file
                    valid_path="valid.csv", # Path to save the validation CSV file
                    data_save_path=None, # Path to save the data files
                    ):
    """
    Splits a DataFrame or CSV file into train, test, and optional validation sets.

    """
    # Load data
    if isinstance(input_data, str):
        df = pd.read_csv(input_data)
    elif isinstance(input_data, pd.DataFrame):
        df = input_data.copy()
    else:
        raise ValueError("input_data must be a path to a CSV file or a DataFrame")
    
    if data_save_path:
        train_path = os.path.join(data_save_path, train_path)
        test_path = os.path.join(data_save_path, test_path)
        valid_path = os.path.join(data_save_path, valid_path)

    # Check if split_column exists and has "train", "test", or "validation" values
    if split_column and split_column in df.columns:
        # Use pre-defined split values if available
        train_df = df[df[split_column] == "train"].copy()
        test_df = df[df[split_column] == "test"].copy()
        valid_df = df[df[split_column] == "validation"].copy() if "validation" in df[split_column].unique() else None
    else:
        # Otherwise, calculate test and validation splits based on fractions
        test_fraction = 1.0 - train_fraction - valid_fraction
        if test_fraction <= 0:
            raise ValueError("train_fraction and valid_fraction must sum to less than 1.")

        # Randomly split data
        train_df, temp_df = train_test_split(df, test_size=(1 - train_fraction), stratify=df[split_column] if stratify and split_column else None)
        train_df = train_df.copy()
        temp_df = temp_df.copy()

        if valid_fraction > 0:
            valid_size = valid_fraction / (valid_fraction + test_fraction)
            valid_df, test_df = train_test_split(temp_df, test_size=(1 - valid_size), stratify=temp_df[split_column] if stratify and split_column else None)
            valid_df = valid_df.copy()
            test_df = test_df.copy()
        else:
            test_df = temp_df
            valid_df = None

    # Optionally add 'is_valid' column in train set if valid_fraction > 0
    if add_is_valid and valid_fraction > 0:
        train_df.loc[:, 'is_valid'] = 0  # Avoid SettingWithCopyError by using .loc
        if valid_df is None:
            # Sample validation rows from the training set if no pre-defined validation split
            valid_indices = train_df.sample(frac=valid_fraction / train_fraction).index
            train_df.loc[valid_indices, 'is_valid'] = 1
        else:
            print(f"'is_valid' column added to '{train_path}' for validation samples within the training set.")
    elif valid_df is not None:
        # Save validation set as a separate CSV if not adding 'is_valid' in training set
        valid_df.to_csv(valid_path, index=False)
        print(f"Validation file saved as '{valid_path}'.")

    # Save train and test sets as CSV files
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    
    print(f"Train and test files saved as '{train_path}' and '{test_path}' respectively.")
    if add_is_valid and valid_fraction > 0:
        print(f"'is_valid' column added to '{train_path}' for validation samples.")


In [ ]:
#| export 
def add_columns_to_csv(csv_path, # Path to the input CSV file
                       column_data, # Dictionary of column names and values to add. Each value can be a scalar (single value for all rows) or a list matching the number of rows.
                       output_path=None, # Path to save the updated CSV file. If None, it overwrites the input CSV file.
                       ):
    """
    Adds one or more new columns to an existing CSV file.

    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Iterate over each column and add to the DataFrame
    for column_name, column_values in column_data.items():
        # Check if column_values is a list and matches DataFrame length
        if isinstance(column_values, list) and len(column_values) != len(df):
            raise ValueError(f"Length of values for column '{column_name}' does not match the number of rows in the CSV.")
        
        # Add the new column
        df[column_name] = column_values

    # Save the updated DataFrame to a CSV file
    output_path = output_path or csv_path
    df.to_csv(output_path, index=False)

    print(f"Columns {list(column_data.keys())} added successfully. Updated file saved to '{output_path}'")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()